In [ ]:
# Create a method to split the dataset into train and test set
def split_dataset(dataset, train_ratio=0.8):
	"""
	Function that can be used to split the dataset into train and test set.
	Args:
		dataset (DeepLontarDataset): Dataset to split
		train_ratio (float): Ratio of the dataset to use for training
	Returns:
		train_dataset (DeepLontarDataset): Training dataset
		test_dataset (DeepLontarDataset): Test dataset
	"""
	train_size = int(train_ratio * len(dataset))
	test_size = len(dataset) - train_size
	train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
	return train_dataset, test_dataset

# Create a class that Implements YOLOv3 model architecture
# The input model is the image data and its annotations (bounding boxes) and class_id in format [xmin, ymin, xmax, ymax, class_id]
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch.autograd import Variable
from __future__ import division

def parse_cfg(cfg_file):
	"""
	Takes a configuration file.

	Returns a list of blocks. Each blocks describes a block in the neural
	network to be built. Block is represented as a dictionary in the list.
	Source: https://blog.paperspace.com/how-to-implement-a-yolo-v3-object-detector-from-scratch-in-pytorch-part-2/
	"""
	with open(cfg_file, 'r') as f:
		lines = f.read().split('\n')
		lines = [x for x in lines if len(x) > 0]
		lines = [x for x in lines if x[0] != '#']
		lines = [x.rstrip().lstrip() for x in lines]
	
	block = {}
	blocks = []
	
	from line in lines:
		if line[0] == "[":				# This marks the start of a new block
			if len(block) != 0:			# If block is not empty, implies it is storing values of previous block.
				blocks.append(block)	# add it the blocks list
				block = {}				# re-init the block
			block["type"] = line[1:-1].rstrip()
		else:
			key, value = line.split("=")
			block[key.rstrip()] = value.lstrip()

	blocks.append(block)
	return blocks
	
def create_modules(blocks):
	"""
	Now we are going to use the list returned 
	by the above parse_cfg to construct PyTorch modules 
	for the blocks present in the config file.

	The create_modules function takes a list blocks returned by the parse_cfg function.
	"""
	net_info = blocks[0]	# Captures the information about the input and pre-processing
	module_list = nn.ModuleList()
	prev_filters = 3
	output_filters = []

	for index, x in enumerate(blocks[1:]):
		module = nn.Sequential()

		# check the type of block
		# create a new module for the block
		# append to module_list
		if (x['type'] == 'convolutional'):
			# Get the info about the layer
			activation = x['activation']
			try:
				batch_normalize = int(x['batch_normalize'])
				bias = False
			except:
				batch_normalize = 0
				bias = True
			
			filters = int(x['filters'])
			kernel_size = int(x['size'])
			stride = int(x['stride'])
			pad = (kernel_size - 1) // 2 if int(x['pad']) else 0
			
			# Add the convolutional layer
			conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias=bias)
			module.add_module('conv_{0}'.format(index), conv)

			# Add the Batch Norm Layer
			if batch_normalize:
				bn = nn.BatchNorm2d(filters)
				module.add_module('batch_norm_{0}'.format(index), bn)

			# Check the activation.
			# It is either Linear or a Leaky ReLU for YOLO
			if activation == 'leaky':
				activn = nn.LeakyReLU(0.1, inplace=True)
				module.add_module('leaky_{0}'.format(index), activn)

		# If it's an upsampling layer
		# We use Bilinear2dUpsampling
		elif (x['type'] == 'upsample'):
			stride = int(x['stride'])
			upsample = nn.Upsample(scale_factor=2, mode='bilinear')
			module.add_module('upsample_{}'.format(index), upsample)
		
		# If it is a route layer
		elif (x['type'] == 'route'):
			x['layers'] = x['layers'].split(',')
			# Start  of a route
			start = int(x['layers'][0])
			# end, if there exists one.
			try:
				end = int(x['layers'][1])
			except:
				end = 0
			# Positive annotation
			if start > 0: 
				start = start - index
			
			if end > 0: 
				end = end - index
			
			route = EmptyLayer()
			module.add_module('route_{0}'.format(index), route)
			if end < 0:
				filters = output_filters[index + start] + output_filters[index + end]
			else:
				filters = output_filters[index + start]
				
		# shortcut corresponds to skip connection
		elif x['type'] == 'shortcut':
			shortcut = EmptyLayer()
			module.add_module('shortcut_{}'.format(index), shortcut)

class EmptyLayer(nn.Module):
	def __init__(self):
		super(EmptyLayer, self).__init__()

class DetectionLayer(nn.Module):
	def __init__(self, anchors):
		super(DetectionLayer, self).__init__()
		self.anchors = anchors


